In [5]:
from pyspark.sql import Row
from difflib import SequenceMatcher
import numpy as np
import pandas as pd

def read_bilivideo(textpath):
    sc = spark.sparkContext
    # Load a text file and convert each line to a Row.
    lines = sc.textFile(textpath)
    parts = lines.map(lambda l: l.split(",")).filter(lambda line : len(line)==28)
    # Load a text file and convert each line to a Row.
    video= parts.map(lambda p: Row(video_url=p[0], video_code=int(p[1]), title=str(p[2])[2:-1], publish_time=p[3], category1=p[4], category2=p[5], video_uploader=p[6],
                                  uploader_id=p[7], uploader_upload_videos=p[8], up_sex=p[9], up_birth=p[10], up_fans=p[11],
                                  up_friend=p[12], up_attention=p[13], up_level=p[14], up_plate=p[15], up_pla_level=p[16],up_pla_condition=[17],
                                  view=p[18],danmaku=p[19],reply=p[20],favorite=p[21],coin=p[22],share=p[23],now_rank=p[24],his_rank=p[25],like_num=p[26],crawl_time=p[27])) #
    videoDF = spark.createDataFrame(video)
    return videoDF



def read_acfunvideo(textpath):
    sc = spark.sparkContext
    # Load a text file and convert each line to a Row.
    lines = sc.textFile(textpath)
    parts = lines.map(lambda l: l.split(",")).filter(lambda line : len(line)==16)
    # Load a text file and convert each line to a Row.
    video_2= parts.map(lambda p: Row(video_url=p[0], title=p[1], video_code=int(p[2]), data_view=p[3], data_collect=p[4], data_comment=p[5], data_vid=p[6],
                                  category_1=p[7], category_2=p[8], publish_time=p[9], bananas=p[10], data_from=p[11],
                                  us_name=p[12], data_uid=p[13], description=p[14], crawl_time=p[15]))
    video_2DF = spark.createDataFrame(video_2)
    return video_2DF

bivideoDF = read_bilivideo("Bvideo1.log")
acvideoDF = read_acfunvideo("Acfunvideo1.log")
# acvideoDF.toPandas().head()

In [15]:
def CompareRatio(texta,textb):
    ratio = SequenceMatcher(None, texta, textb).ratio()
    if ratio > 0.66:
        return 1,ratio
    else:
        return 0,ratio
VecCompare = np.vectorize(CompareRatio)


In [7]:
biTitleList = bivideoDF.select("title").rdd.flatMap(lambda x: x).collect()
acTitleList = acvideoDF.select("title").rdd.flatMap(lambda x: x).collect()
biCodeList = bivideoDF.select("video_code").rdd.flatMap(lambda x: x).collect()
acCodeList = acvideoDF.select("video_code").rdd.flatMap(lambda x: x).collect()
len(acCodeList)

315259

In [16]:
fullMatchList = []
import timeit

start = timeit.default_timer()
for i in range(15): #len(acTitleList)
    forindex,ratioList1 = VecCompare(acTitleList[i],biTitleList)
    index = np.nonzero(forindex)
    for j in index[0]:
        fullMatchList.append((acCodeList[i],acTitleList[i],biCodeList[j],biTitleList[j],ratioList1[j]))
        
print('length of the whole matched list is '+str(len(fullMatchList)))
stop = timeit.default_timer()
timerun = str(stop - start)
print ('running this step costs '+timerun+' seconds')

length of the whole matched list is 18
running this step costs 226.6828520940942 seconds


In [18]:
labels = ['aCode', 'aTitle', 'bCode', 'bTitle', 'similarity']
Matchdf = pd.DataFrame.from_records(fullMatchList, columns=labels) 
Matchdf


,aCode,aTitle,bCode,bTitle,similarity
0,2000011,'2015-2016动画电影前瞻',6016725,2015-2016,0.666667
1,2000038,'★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第四集》',6061568,★我的世界★Minecraft《籽岷的多人模组极限生存挑战 2015万圣节 第三集》,0.734177
2,2000038,'★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第四集》',6061636,★我的世界★Minecraft《籽岷的多人模组极限生存挑战 2015万圣节 第四集》,0.759494
3,2000038,'★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第四集》',6077779,★我的世界★Minecraft《籽岷的多人模组极限生存挑战 2015万圣节 第五集》,0.734177
4,2000038,'★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第四集》',6120968,★我的世界★Minecraft《籽岷的多人模组高难度生存挑战 奔跑吧2016 第二集》,0.725000
5,2000038,'★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第四集》',6120979,★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第五十集》,0.916667
6,2000038,'★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第四集》',6121005,★我的世界★Minecraft《籽岷的多人模组高难度生存挑战 奔跑吧2016 第一集》,0.725000
7,2000038,'★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第四集》',6148955,★我的世界★Minecraft《籽岷的多人模组高难度生存挑战 奔跑吧2016 第四集》,0.750000
8,2000038,'★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第四集》',6148963,★我的世界★Minecraft《籽岷的多人模组高难度生存挑战 奔跑吧2016 第三集》,0.725000
9,2000038,'★我的世界★Minecraft《籽岷的模组生存 虚无世界2 第四集》',6148966,★我的世界★Minecraft《籽岷的多人模组高难度生存挑战 奔跑吧2016 第七集》,0.725000
